In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install rasterio  -q -q -q
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q

In [18]:
!pip install scikit-image

In [3]:
import sys
sys.path.append('../src')
from classes.data.satellite_image import SatelliteImage
from utils.utils import *
from utils.plot_utils import *
from utils.image_utils import *
import utils.mappings as mapps

In [4]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import re
from pyproj import Transformer
from datetime import date
from scipy.ndimage import label
import time
from tqdm import tqdm
import os

In [7]:
update_storage_access()

In [8]:
environment = get_environment()

root_path = get_root_path()

bucket = environment["bucket"]

#2022
#guyane 2022
path_s3_pleiades_data_2022_guyane = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_pleiades_data_2022_guyane = environment["local-path"]["PLEIADES"][2022]["guyane"]

#martinique 2022
path_s3_pleiades_data_2022_martinique = environment["sources"]["PLEIADES"][2022]["martinique"]
path_local_pleiades_data_2022_martinique = environment["local-path"]["PLEIADES"][2022]["martinique"]
          
#guadeloupe 2022
path_s3_pleiades_data_2022_guadeloupe = environment["sources"]["PLEIADES"][2022]["guadeloupe"]
path_local_pleiades_data_2022_guadeloupe = environment["local-path"]["PLEIADES"][2022]["guadeloupe"]

#mayotte 2022
path_s3_pleiades_data_2022_mayotte = environment["sources"]["PLEIADES"][2022]["976"]
path_local_pleiades_data_2022_mayotte = environment["local-path"]["PLEIADES"][2022]["976"]

#2020
#mayotte 2020
path_s3_pleiades_data_2020_mayotte = environment["sources"]["PLEIADES"][2020]["976"]
path_local_pleiades_data_2020_mayotte = environment["local-path"]["PLEIADES"][2020]["976"]

#2019
#mayotte 2019
path_s3_pleiades_data_2019_mayotte = environment["sources"]["PLEIADES"][2019]["976"]
path_local_pleiades_data_2019_mayotte = environment["local-path"]["PLEIADES"][2019]["976"]

#2018
#reunion 2018
path_s3_pleiades_data_2018_reunion = environment["sources"]["PLEIADES"][2018]["reunion"]
path_local_pleiades_data_2018_reunion = environment["local-path"]["PLEIADES"][2018]["reunion"]

#2017
#martinique 2017
path_s3_pleiades_data_2017_martinique = environment["sources"]["PLEIADES"][2017]["martinique"]
path_local_pleiades_data_2017_martinique = environment["local-path"]["PLEIADES"][2017]["martinique"]

#mayotte 2017
path_s3_pleiades_data_2017_mayotte = environment["sources"]["PLEIADES"][2017]["976"]
path_local_pleiades_data_2017_mayotte = environment["local-path"]["PLEIADES"][2017]["976"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [9]:
environment = get_environment()
root_path = get_root_path()
bucket = environment["bucket"]

#mayotte 2022
path_s3_pleiades_data_2022_mayotte = environment["sources"]["PLEIADES"][2022]["976"]
path_local_pleiades_data_2022_mayotte = environment["local-path"]["PLEIADES"][2022]["976"]

#2020
#mayotte 2020
path_s3_pleiades_data_2020_mayotte = environment["sources"]["PLEIADES"][2020]["976"]
path_local_pleiades_data_2020_mayotte = environment["local-path"]["PLEIADES"][2020]["976"]

#2019
#mayotte 2019
path_s3_pleiades_data_2019_mayotte = environment["sources"]["PLEIADES"][2019]["976"]
path_local_pleiades_data_2019_mayotte = environment["local-path"]["PLEIADES"][2019]["976"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

In [10]:
# DL PLEIADE mayotte 2020
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2020_mayotte}",
        lpath=f"../{path_local_pleiades_data_2020_mayotte}",
        recursive=True)

In [ ]:
# DL PLEIADE mayotte 2017
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_mayotte}",
        lpath=f"../{path_local_pleiades_data_2017_mayotte}",
        recursive=True)

In [9]:
# DL PLEIADE martinique 2022 utilisé !
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_martinique}",
        lpath=f"../{path_local_pleiades_data_2022_martinique}",
        recursive=True)

In [57]:
# DL PLEIADE martinique 2017 utilisé !
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2017_martinique}",
        lpath=f"../{path_local_pleiades_data_2017_martinique}",
        recursive=True)

In [6]:
# DL PLEIADE guyane 2022 utilisé !
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_guyane}",
        lpath=f"../{path_local_pleiades_data_2022_guyane}",
        recursive=True)

En martinique :
(14.635338, -61.038345) plateau médico-technique construit en 2018 (côté droit du triangle). 
Essayons de retrouver l'image qui le contient.

In [210]:
#Nouvelle image 1
filename_1 = find_image_of_point([14.635338, -61.038345], '../data/PLEIADES/2022/MARTINIQUE', coord_gps = True)
date_1 = date.fromisoformat('2022-01-01')

In [129]:
image_1 = SatelliteImage.from_raster(
        filename_1,
        date = date_1, 
        n_bands = 3,
        dep = "972"
    )
image_1.normalize()

In [130]:
image_1.plot([0,1,2]) 

In [97]:
#image 2
filename_2 = find_image_different_years(2017, None, filename_1)
date_2 = date.fromisoformat('2017-01-01')

In [98]:
image_2 = SatelliteImage.from_raster(
        filename_2,
        date = date_2, 
        n_bands = 3,
        dep = "972"
    )
image_2.normalize()

In [89]:
image_2.plot([0,1,2])

On a bien un changement à droite du triangle. Il ne reste plus qu'a avoir la bande infrarouge pour faire les masques puis les différences.

On va chercher en guyane : (-12.755601, 45.229945) coordonnées d'un quartier

In [31]:
filename_5 = find_image_of_point([-12.755601, 45.229945], '../data/PLEIADES/2020/MAYOTTE', coord_gps = True)
date_5 = date.fromisoformat('2020-01-01')
print(filename_5)

In [32]:
image_5 = SatelliteImage.from_raster(
        filename_5,
        date = date_5, 
        n_bands = 3,
        dep = "976"
    )

In [33]:
image_5.plot([0,1,2])
image_5 = SatelliteImage.from_raster(
        filename_5,
        date = date_5, 
        n_bands = 3,
        dep = "976"
    )

In [39]:
from utils.gestion_label_studio import create_mask_from_label_studio_export

create_mask_from_label_studio_export(
            num_task = "3",
            name_output = "mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits"
        )

mask  = np.load("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy")
plt.imshow(np.transpose(mask), cmap="gray", interpolation="nearest")

np.save("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy",np.transpose(mask))

mc cp s3/projet-slums-detection/Donnees/PLEIADES/MAYOTTE/2020/MOSA/ORT_2020052526670967_0524_8590_U38S_8Bits.jp2 s3/projet-slums-detection/donnees-test/segmentation/images/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.jp2
mc cp export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy s3/projet-slums-detection/donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy


In [148]:
filename_6 = find_image_different_years(2017, image_5)
date_6 = date.fromisoformat('2020-01-01')

In [149]:
image_6 = SatelliteImage.from_raster(
        filename_6,
        date = date_6, 
        n_bands = 3,
        dep = "976"
    )
image_6.normalize()

In [150]:
image_6.plot([0,1,2])

Dans la moitié haute de l'image à droite, il y a plusieurs bâtiments qui ont été construits.

Pas de RIL pour ces deux années en mayotte ...

In [15]:
filename_7 = find_image_of_point([-12.753436, 45.219888], '../data/PLEIADES/2020/MAYOTTE', coord_gps = True)
date_7 = date.fromisoformat('2020-01-01')
print(filename_7)

In [105]:
image_7 = SatelliteImage.from_raster(
        filename_7,
        date = date_7, 
        n_bands = 3,
        dep = "976"
    )
image_7.normalize()

In [38]:
image_7.plot([0,1,2])

In [30]:
from utils.gestion_label_studio import create_mask_from_label_studio_export

create_mask_from_label_studio_export(
            num_task = "1",
            name_output = "mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit"
        )


mc cp s3/projet-slums-detection/Donnees/PLEIADES/MAYOTTE/2020/MOSA/ORT_2020052526670967_0523_8591_U38S_8Bits.jp2 s3/projet-slums-detection/donnees-test/segmentation/images/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bits.jp2

mc cp export/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy s3/projet-slums-detection/donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy


In [106]:
from PIL import Image
import numpy as np
import io

# Créer un tableau avec des données d'exemple
data = image_7.array
data = (data * 255).astype(np.uint8)
data = data.transpose(1,2,0)


# Convertir le tableau en image
image = Image.fromarray(data)

# Sauvegarder l'image en tant que fichier PNG
image.save("image.png")

# Enregistrer l'image en tant qu'objet bytes pour téléchargement
img_bytes = io.BytesIO()
image.save(img_bytes, format='PNG')
img_bytes.seek(0)

# Envoyer l'image pour téléchargement
from IPython.display import FileLink
display(FileLink("image.png"))

In [7]:
filename_8 = find_image_of_point([-12.753436, 45.219888], '../data/PLEIADES/2017/MAYOTTE', coord_gps = True)
#filename_8 = find_image_different_years(2017, image_7)
date_8 = date.fromisoformat('2017-01-01')

In [103]:
image_8 = SatelliteImage.from_raster(
        filename_8,
        date = date_8, 
        n_bands = 3,
        dep = "976"
    )
image_8.normalize()

In [101]:
image_8.plot([0,1,2])

In [104]:
from PIL import Image
import numpy as np
import io

# Créer un tableau avec des données d'exemple
data = image_8.array
data = (data * 255).astype(np.uint8)
data = data.transpose(1,2,0)


# Convertir le tableau en image
image = Image.fromarray(data)

# Sauvegarder l'image en tant que fichier PNG
image.save("image.png")

# Enregistrer l'image en tant qu'objet bytes pour téléchargement
img_bytes = io.BytesIO()
image.save(img_bytes, format='PNG')
img_bytes.seek(0)

# Envoyer l'image pour téléchargement
from IPython.display import FileLink
display(FileLink("image.png"))

In [12]:
data = image_8.array.copy()
data = (data * 255).astype(np.uint8)
data = data.transpose(1,2,0)

In [14]:
plt.imshow(data)
plt.axis('off')  # Turn off axis labels
plt.show()

In [ ]:
from PIL import Image
import numpy as np
import io

# Créer un tableau avec des données d'exemple
data = image_8.array
data = (data * 255).astype(np.uint8)
data = data.transpose(1,2,0)


# Convertir le tableau en image
image = Image.fromarray(data)

# Sauvegarder l'image en tant que fichier PNG
image.save("image.png")

# Enregistrer l'image en tant qu'objet bytes pour téléchargement
img_bytes = io.BytesIO()
image.save(img_bytes, format='PNG')
img_bytes.seek(0)

# Envoyer l'image pour téléchargement
from IPython.display import FileLink
display(FileLink("image.png"))

In [42]:
data2 = image_7.array.copy()
data2 = data2.transpose(1,2,0)

In [43]:
plt.imshow(data2)

In [54]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(data)
ax1.set_title("Image 2017")
ax1.axis('off')
ax2.imshow(data2)
ax2.set_title("Image 2020")
ax2.axis('off')
plt.show()


In [108]:
from classes.labelers.labeler import BDTOPOLabeler

In [111]:
labeler_BDTOPO2020 = BDTOPOLabeler(date_7, '976')

In [112]:
labeler_BDTOPO2017 = BDTOPOLabeler(date_8, '976')

In [114]:
mask_BDTOPO2020 = labeler_BDTOPO2020.create_segmentation_label(image_7)

In [115]:
mask_BDTOPO2017 = labeler_BDTOPO2017.create_segmentation_label(image_8)

In [117]:
plt.imshow(mask_BDTOPO2020, cmap="gray")

In [118]:
plt.imshow(mask_BDTOPO2017, cmap="gray")

In [223]:
import pandas as pd

xmin, ymin, xmax, ymax = image_7.bounds
patch_2017 = labeler_BDTOPO2017.labeling_data.cx[xmin:xmax, ymin:ymax].copy()
patch_2020 = labeler_BDTOPO2020.labeling_data.cx[xmin:xmax, ymin:ymax].copy()

common_crs = "EPSG:4471"

# Transformer les géométries en utilisant le CRS commun
patch_2020 = patch_2020.to_crs(common_crs)
patch_2017 = patch_2017.to_crs(common_crs)


gdf = pd.concat([patch_2020, patch_2017], ignore_index=True)

# Filtrer le GeoDataFrame pour exclure les ID en double et leurs doublons
patch = gdf[~gdf['ID'].duplicated(keep=False)]


if patch.empty:
    rasterized10 = np.zeros(
        image_7.array.shape[1:], dtype=np.uint8
    )
else:
    rasterized10 = rasterize(
        patch.geometry,
        out_shape=image_7.array.shape[1:],
        fill=0,
        out=None,
        transform=image_7.transform,
        all_touched=True,
        default_value=1,
        dtype=None,
    ) 

In [211]:
xmin, ymin, xmax, ymax = image_7.bounds
patch_2017 = labeler_BDTOPO2017.labeling_data.cx[xmin:xmax, ymin:ymax].copy()
patch_2020 = labeler_BDTOPO2020.labeling_data.cx[xmin:xmax, ymin:ymax].copy()

common_crs = "EPSG:4471"

# Transformer les géométries en utilisant le CRS commun
patch_2020 = patch_2020.to_crs(common_crs)
patch_2017 = patch_2017.to_crs(common_crs)


join_result = gpd.sjoin(patch_2020, patch_2017, how='inner', predicate='contains')

# Identifier les bâtiments qui ne sont pas dans la jointure
patch = patch_2017.loc[~patch_2017.index.isin(join_result.index_right)]


if patch.empty:
    rasterized = np.zeros(
        image_7.array.shape[1:], dtype=np.uint8
    )
else:
    rasterized = rasterize(
        patch.geometry,
        out_shape=image_7.array.shape[1:],
        fill=0,
        out=None,
        transform=image_7.transform,
        all_touched=True,
        default_value=1,
        dtype=None,
    ) 




In [218]:
join_result = gpd.sjoin(patch_2017, patch_2020, how='inner', predicate='contains')

# Identifier les bâtiments qui ne sont pas dans la jointure
patch = patch_2020.loc[~patch_2020.index.isin(join_result.index_right)]


if patch.empty:
    rasterized2 = np.zeros(
        image_7.array.shape[1:], dtype=np.uint8
    )
else:
    rasterized2 = rasterize(
        patch.geometry,
        out_shape=image_7.array.shape[1:],
        fill=0,
        out=None,
        transform=image_7.transform,
        all_touched=True,
        default_value=1,
        dtype=None,
    ) 



In [220]:
patch = gpd.sjoin(patch_2017, patch_2020, how='inner', predicate='contains')

if patch.empty:
    rasterized3 = np.zeros(
        image_7.array.shape[1:], dtype=np.uint8
    )
else:
    rasterized3 = rasterize(
        patch.geometry,
        out_shape=image_7.array.shape[1:],
        fill=0,
        out=None,
        transform=image_7.transform,
        all_touched=True,
        default_value=1,
        dtype=None,
    ) 



In [ ]:
import pandas as pd

xmin, ymin, xmax, ymax = image_7.bounds
patch_2017 = labeler_BDTOPO2017.labeling_data.cx[xmin:xmax, ymin:ymax].copy()
patch_2020 = labeler_BDTOPO2020.labeling_data.cx[xmin:xmax, ymin:ymax].copy()

common_crs = "EPSG:4471"

# Transformer les géométries en utilisant le CRS commun
patch_2020 = patch_2020.to_crs(common_crs)
patch_2017 = patch_2017.to_crs(common_crs)


gdf = pd.concat([patch_2020, patch_2017], ignore_index=True)

# Filtrer le GeoDataFrame pour exclure les ID en double et leurs doublons
patch = gdf[~gdf['ID'].duplicated(keep=False)]


if patch.empty:
    rasterized10 = np.zeros(
        image_7.array.shape[1:], dtype=np.uint8
    )
else:
    rasterized10 = rasterize(
        patch.geometry,
        out_shape=image_7.array.shape[1:],
        fill=0,
        out=None,
        transform=image_7.transform,
        all_touched=True,
        default_value=1,
        dtype=None,
    ) 

In [237]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20, 5))
ax1.imshow(np.transpose(image_8.array, (1, 2, 0))[:,:,:3])
ax1.imshow(rasterized3, alpha=0.3)
ax1.set_title("intersect")
ax1.axis('off')
ax2.imshow(np.transpose(image_8.array, (1, 2, 0))[:,:,:3])
ax2.imshow(mask_BDTOPO2017, alpha=0.3)
ax2.set_title("mask 2017")
ax2.axis('off')
ax3.imshow(np.transpose(image_7.array, (1, 2, 0))[:,:,:3])
ax3.imshow(mask_BDTOPO2020, alpha=0.3)
#ax3.imshow(mask_BDTOPO2017, alpha=0.3)
ax3.set_title("mask 2020")
ax3.axis('off')
plt.show()

In [222]:
fig, (ax1, ax2, ax3, ax4,ax5) = plt.subplots(1, 5, figsize=(25, 5))
ax1.imshow(rasterized, cmap="gray")
ax1.set_title("hors intersection 2017")
ax1.axis('off')
ax2.imshow(mask_BDTOPO2017, cmap="gray")
ax2.set_title("mask 2017")
ax2.axis('off')
ax3.imshow(mask_BDTOPO2020, cmap="gray")
ax3.set_title("mask 2020")
ax3.axis('off')
ax4.imshow(rasterized2, cmap="gray")
ax4.set_title("hors intersection 2020")
ax4.axis('off')
ax5.imshow(rasterized3, cmap="gray")
ax5.set_title("intersection 2017 2020")
ax5.axis('off')
plt.show()

In [158]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_7.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_BDTOPO2020, alpha=0.3)

In [159]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_8.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_BDTOPO2017, alpha=0.3)

In [160]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_8.array, (1, 2, 0))[:,:,:3])
ax.imshow(rasterized, alpha=0.3)